# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for AAPL to identify trends and predict market movements in order to provide valuable insights and alerts.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements to provide insights and alerts about significant opportunities or threats for AAPL."}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL to analyze trends and predict market movements using statistical modeling and machine learni

Final Answer: The current stock price for Apple Inc. (AAPL) is $236.00 with a decrease of 0.67% at close on January 31 at 4:00:02 PM EST. The stock has shown a 5.93% increase in the last month, a 6.42% decrease year-to-date, and a 25.51% increase over the past year. The company's market cap is 3.545T, with a PE ratio of 37.46 and an EPS of 6.30. Apple Inc. operates in the Consumer Electronics/Technology sector and offers a range of products including smartphones, personal computers, tablets, wearables, and accessories. Additionally, the company provides various subscription-based services such as Apple Arcade, Apple Fitness+, Apple Music, Apple News+, Apple TV+, and Apple Pay. The company has a strong financial position with a profit margin of 23.97% and a return on equity of 157.41%.

> Finished chain.
 

The current stock price for Apple Inc. (AAPL) is $236.00 with a decrease of 0.67% at close on January 31 at 4:00:02 PM EST. The stock has shown a 5.93% increase in the last month, a 

Final Answer:
Based on the insights gathered from multiple sources, including MarketWatch, Investopedia, Barchart, Emirhan Kabakci, Permutable, Morningstar, Scramble, and Forbes, it is evident that various factors influence Apple Inc.'s stock price. These factors include product launches, free cash flow results, economic indicators (such as GDP growth, inflation rates, and interest rates), new product launches (e.g., iPhone, iPad, Mac), market trends, shareholder returns, and innovation driving market confidence.

To develop and refine trading strategies for AAPL, it is crucial to consider these key factors and trends influencing its stock price. By incorporating statistical modeling and machine learning techniques with insights from the Data Analyst Agent and the information gathered from these sources, we can make informed investment decisions to optimize profitability and mitigate risks in line with the user-defined risk tolerance level.

> Finished chain.
 

Based on the insights g

Final Answer:

Apple Inc. Stock Quote (U.S.: Nasdaq) - AAPL - MarketWatch

Key Data
Open: $247.19
Day Range: $233.44 - $247.19
52 Week Range: $164.08 - $260.10
Market Cap: $3.55T
Shares Outstanding: 15.04B
Public Float: 15.02B

From the search results, we can see that AAPL is currently trading near the top of its 52-week range and above its 200-day simple moving average. The current price of AAPL is $236.00 USD, with a decrease of -0.67% in the past 24 hours. The average price target is $252.86, with a high forecast of $324.00 and a low forecast of $188.00. 

Based on these current market conditions and pricing information, we can now develop and test detailed execution plans for the Momentum Trading Strategy for AAPL.

> Finished chain.
 

Apple Inc. Stock Quote (U.S.: Nasdaq) - AAPL - MarketWatch

Key Data
Open: $247.19
Day Range: $233.44 - $247.19
52 Week Range: $164.08 - $260.10
Market Cap: $3.55T
Shares Outstanding: 15.04B
Public Float: 15.02B

From the search results, we can see 

Final Answer: 
After analyzing the current trading situation of AAPL on Yahoo Finance, it is evident that the stock has shown a YTD return of -6.42% and a 1-year return of 25.51%. The proposed Momentum Trading Strategy for AAPL involves waiting for it to break above its 200-day simple moving average and the upper Bollinger Band to confirm an uptrend, selling when the price starts to decline and falls below the 200-day simple moving average or the lower Bollinger Band, setting a stop-loss at 5% below the entry point, taking profits when AAPL reaches $324.00 or shows signs of a reversal, and continuously monitoring price movements.

Potential Risks:
1. Market Volatility: The stock market can experience sudden fluctuations, impacting the effectiveness of the trading strategy.
2. False Breakouts: AAPL may temporarily break above the moving average and upper Bollinger Band, only to reverse quickly, leading to losses.
3. Stop-Loss Activation: Setting a stop-loss at 5% below the entry point m

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

After analyzing the risks associated with the proposed Momentum Trading Strategy for AAPL, it is evident that there are several potential challenges that could impact the success of the strategy. Market volatility, false breakouts, premature stop-loss activation, and timing the reversal point are key risks that need to be addressed.

To mitigate these risks, it is recommended to diversify the investment portfolio to spread risk exposure, use additional technical indicators for confirmation, adjust the stop-loss level based on intraday price movements, and continuously monitor market conditions to adapt the strategy accordingly.

By implementing these mitigation strategies and staying informed about market developments, we can optimize profitability and make informed investment decisions based on the current market conditions and pricing information.